In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

#needed for jupyter notebook
#https://github.com/tensorflow/tensorflow/issues/24828
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Num GPUs Available:  3


In [2]:
import os

import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image

MODEL_PATH = "saved_models"
net_file = "120cs_3_15_split_9947_1584571828_RES152V2.h5"

net_path = os.path.join(MODEL_PATH, net_file)
net = load_model(net_path)
print("Loaded:", net_path)

Loaded: saved_models/120cs_3_15_split_9947_1584571828_RES152V2.h5


In [11]:
TARGET_SIZE = (224,112) #make sure this matches
DATASET = "3_15_split"
IMG_DIR = os.path.join(DATASET,"train")
ALL_CLASSES = sorted(os.listdir(os.path.join(DATASET,"train")))
if ".DS_Store" in ALL_CLASSES:
    ALL_CLASSES.remove(".DS_Store")

In [12]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

imgs = []
classes = []
for cls in ALL_CLASSES: #class is Python keyword
    print("-"*5,cls,"-"*5,"\n")
    CLASS_DIR = os.path.join(IMG_DIR, cls)
    for filename in os.listdir(CLASS_DIR):
        img_path = os.path.join(CLASS_DIR, filename)

        img = image.load_img(img_path, target_size=TARGET_SIZE)
        if not img: continue
        x = image.img_to_array(img)
        x = preprocess_input(x)
#         x = np.expand_dims(x, axis=0) #need to add dim to put into resnet

        imgs.append(x)
        classes.append(ALL_CLASSES.index(cls))
#         print(len(imgs))

imgs = np.asarray(imgs)
classes = np.asarray(classes)

print(imgs.shape)
print(classes.shape)
print(classes)
Y_pred = net.predict(imgs)
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)
print('Confusion Matrix')
print(confusion_matrix(classes, y_pred))

----- black_bishop ----- 

----- black_king ----- 

----- black_knight ----- 

----- black_pawn ----- 

----- black_queen ----- 

----- black_rook ----- 

----- empty ----- 

----- white_bishop ----- 

----- white_king ----- 

----- white_knight ----- 

----- white_pawn ----- 

----- white_queen ----- 

----- white_rook ----- 

(17406, 224, 112, 3)
(17406,)
[ 0  0  0 ... 12 12 12]
[ 0  0  0 ... 12 12 12]
Confusion Matrix
[[ 605    0    1    1    1    1    0    0    0    0    0    0    0]
 [   0  574    0    0    4    2    0    0    0    0    0    0    0]
 [   0    1  464    0    1    0    1    0    0    0    0    0    0]
 [   1    0    0 2953    2    0    3    0    0    0    3    0    0]
 [   0    5    0    0  413    0    0    0    0    0    0    0    0]
 [   0    0    0    3    0  668    0    0    0    0    0    0    0]
 [   0    0    0    1    0    3 5913    0    0    1    1    0    0]
 [   0    0    0    0    0    0    2  584    0    0    2    0    0]
 [   0    0    0    0    0    0

In [13]:
# https://github.com/jkjung-avt/keras-cats-dogs-tutorial/blob/master/predict_resnet50.py

"""
loops through every class of a dir like data/valid or data/train
"""
for cls in ALL_CLASSES: #class is Python keyword
    
    if cls != "white_queen": continue
    
    print("-"*5,cls,"-"*5,"\n")
    CLASS_DIR = os.path.join(IMG_DIR, cls)
    for filename in os.listdir(CLASS_DIR):
        img_path = os.path.join(CLASS_DIR, filename)

        img = image.load_img(img_path, target_size=TARGET_SIZE)
        if not img: continue
        x = image.img_to_array(img)
        x = preprocess_input(x)
        x = np.expand_dims(x, axis=0) #need to add dim to put into resnet

        pred = net.predict(x)[0]
        top_inds = pred.argsort()[::-1]
 
        # show if prediction less than 50% sure OR class is wrong
        if pred[top_inds[0]] < 0.5 or ALL_CLASSES[top_inds[0]] != cls:     
            print(img_path)
            for i in top_inds[:5]: #only show top five
                print('{}: {}'.format(ALL_CLASSES[i], pred[i]))
            print()
    


----- white_queen ----- 

3_15_split/train/white_queen/41_high_angle_8.55.26.jpg
white_queen: 0.2997586131095886
white_pawn: 0.1756194531917572
white_bishop: 0.17295777797698975
black_king: 0.1332138180732727
empty: 0.06903279572725296

3_15_split/train/white_queen/48_IMG_9333.jpg
white_knight: 0.9999964237213135
white_queen: 3.399677552806679e-06
white_rook: 4.792164887135186e-08
white_bishop: 4.523374030895866e-08
black_bishop: 4.189865521198044e-08

